In [25]:
#This code is to filter non-specific binding events from the data after running the neural network clustering algorithm
#Files needed are the 'DataCalled' files in the folder 'evaluated_by_?????' and 'ClusterStars' in the folder 'ROIs_????_whole_image'

In [ ]:
import pandas as pd
import numpy as np
import h5py
import glob
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler

file_loc=list([f'./image_{i}_filtered_DataCalled.csv' for i in range(1,9)])   
file_loc_2=list([f'./image_{l}_filtered_ClusterStats.csv' for l in range(1,9)])
result_loc=list([f'image_{i}' for i in range(1,9)])
file_loc, file_loc_2

In [34]:
    def loop_function(file_name,file_name_2,result_name,threshold_mean_lower,threshold_mean_upper,threshold_std):
    
    #Function to filter non-specific binding events using the mean frame
    def filter_func(x):
            return  threshold_mean_lower <  x['frame'].mean() < threshold_mean_upper
    
    #Function to filter non-specific binding events using the std frame
    def filter_func_2(x):
            return  x['frame'].std() > threshold_std
    
    #Reading files from the cluster analysis (DataCalled and ClusterStats)
    df_initial=pd.read_csv(file_name, delimiter=',')
    df_initial_2=pd.read_csv(file_name_2, delimiter=',')
    
    #Filtering non-specific binding events using mean frame 
    df2= df_initial[["frame","clusterID","x","y"]]
    group = df2.groupby('clusterID')
    df3 = group.filter(filter_func)
    df4 = df3.groupby('clusterID')
    fig_1=sns.displot(group.mean(), x="frame", kde=True) 
    fig_2=sns.displot(df4.mean(), x="frame", kde=True) 
    fig_1.savefig(f'./{result_name}_Meanframe_before.jpg')
    fig_1.savefig(f'./{result_name}_Meanframe_after.jpg')
    
    #Filtering non-specific binding events using std frame
    df5 = df4.filter(filter_func_2)
    df6=df5.groupby('clusterID')
    fig_3=sns.displot(group.std(), x="frame", kde=True) 
    fig_4=sns.displot(df6.std(), x="frame", kde=True) 
    fig_3.savefig(f'./{result_name}_stdframe_before.jpg')
    fig_4.savefig(f'./{result_name}_stdframe_after.jpg')
    
    #Filter receptor clusters from artifical clusters
    df7 = df_initial_2.groupby('ClusterID')
    list_final = list(df5['clusterID'])
    df_ReceptorClusters_final = df_initial_2[df_initial_2['ClusterID'].isin(list_final)]
    df_ArtificialClusters_final = df_initial_2[~df_initial_2['ClusterID'].isin(list_final)]
    points = df_F_final['Points']
    area = df_F_final['Area (um2)']
    density = df_F_final['Density (pts/um2)']
    points_median = points.median()
    area_median = area.median()
    density_median = density.median()
    cluster_number = area.count()
    data = [[area_median, density_median, points_median, cluster_number]]
    stat = pd.DataFrame(data, columns=['Area', 'density','points','clusters'])
    
    #Save output
    stat.to_csv(f'{result_name}_stat.csv', index=False)
    df_ReceptorClusters_final.to_csv(f'{result_name}_result.csv', index=False)
    
    return 0
    
#Loop function to process multiple files
#Values needed are for threshold_mean_lower,threshold_mean_upper,threshold_std
for i,l,j in zip(file_loc,file_loc_2,result_loc):
    loop_function(i,l,j,1000,9000,500)
   
  
#Save all stats processed in one csv file
csv_files = glob.glob('*_stat.{}'.format('csv'))
df_append = pd.DataFrame()
for file in csv_files:
            df_temp = pd.read_csv(file)
            df_append = df_append.append(df_temp, ignore_index=True)
df_append.to_csv('allstats.csv')